<a href="https://colab.research.google.com/github/KarelZe/thesis/blob/main/notebooks/data_preprocessing_mem_reduce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qqq
!pip install modin[all]

In [ ]:
import numpy as np
import modin.pandas as pd
import wandb

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

In [ ]:
file_path_input = "gcs/data/raw/matched_ise_quotes.csv"
file_path_output = "gcs/data/preprocessed/matched_ise_quotes_min_mem_usage.pkl"

In [ ]:
# use gcs fuse to access google cloud storage
# https://stackoverflow.com/a/60450255/5755604
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# mount google cloud stoarge as drive
!mkdir gcs
!gcsfuse thesis-bucket-option-trade-classification gcs

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage. 

	Adapted from: [load data (reduce memory usage) | Kaggle](https://www.kaggle.com/code/gemartin/load-data-reduce-memory-usage/notebook)
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
df = import_data(file_path_input)
df.to_pickle(file_path_output)

dataset_at = wandb.Artifact('data_mem_reduced',type="preprocessed_data")
dataset_at.add_reference(file_path_output)
